# IRIS PASSCAL - RT130 Data Processing - Step 2: Generating parameter files

A set of Jupyter notebooks by Glenn Thompson based on: https://www.passcal.nmt.edu/webfm_send/3035

The new version of rt2ms expects the following directory tree structure:

<pre>
SVC01
├── CONFIG
├── DAYS
├── LOGS
├── MSEED
└── RAW
    ├── RT130-91F8-1.cf
    │   ├── 2018289
    │   │   └── 91F8
    │   └── 2018290
    │       └── 91F8
    ├── RT130-9338-1.cf
    │   ├── 2018289
    │   │   └── 9338
    │   └── 2018290
    │       └── 9338
    ├── RT130-9406-1.cf
    │   ├── 2018289
    │   │   └── 9406
    │   └── 2018290
    │       └── 9406
    ├── RT130-98E6-1.cf
    │   ├── 2018289
    │   │   └── 98E6
    │   └── 2018290
    │       └── 98E6
    ├── RT130-9BC5-1.cf
    │   ├── 2018289
    │   │   └── 9BC5
    │   └── 2018290
    │       └── 9BC5
    ├── RT130-9D7C-1.cf
    │   ├── 2018289
    │   │   └── 9D7C
    │   └── 2018290
    │       └── 9D7C
    └── RT130-AB13-1.cf
        ├── 2018289
        │   └── AB13
        └── 2018290
            └── AB13
SVC02
├── CONFIG
├── DAYS
├── LOGS
├── MSEED
└── RAW
    ├── RT130-91F8-1.cf
    │   ├── 2018290
    │   │   └── 91F8
    │   ├── 2018291
    │   │   └── 91F8
    │   ├── 2018292
    │   │   └── 91F8

    
</pre> etc.

It also needs a parameter file, which can allegedly now be generated by rt2ms with the -e option, e.g. 

<pre>
    ls -d SVCall/RAW/*.cf > dir.list
    rt2ms -D dir.list -e
</pre>

We also want to check this by generating a history of datalogger/station/lat/lon/yyyyjjj

## 2.1 Try to generate parameter files


In [1]:
import os
import libGTpasscal
REFTEKDIR = '/home/thompsong/work/PROJECTS/KSCpasscal'
NEWSVCDIR = os.path.join(REFTEKDIR, '10_ORGANIZED', 'SVCall')
newpaths = libGTpasscal.getpaths(NEWSVCDIR)
def runcmd(cmd):
    print(cmd)
    os.system(cmd)

dirlist = os.path.join(NEWSVCDIR, 'dir.list')
cmd = f"ls -d {newpaths['RAWDIR']}/*.cf > {dirlist}"
runcmd(cmd)

cmd2 = f"rt2ms -D {dirlist} -e"
runcmd(cmd2)

ls -d /home/thompsong/work/PROJECTS/KSCpasscal/10_ORGANIZED/SVCall/RAW/*.cf > /home/thompsong/work/PROJECTS/KSCpasscal/10_ORGANIZED/SVCall/dir.list
rt2ms -D /home/thompsong/work/PROJECTS/KSCpasscal/10_ORGANIZED/SVCall/dir.list -e
2024-11-14 11:46:42 - INFO - rt2ms program starting
2024-11-14 11:46:42 - INFO - Version: 2023.4.2.0
2024-11-14 11:46:42 - WARNING - DeprecationWarning: NotImplemented should not be used in a boolean context
2024-11-14 11:46:42 - WARNING - DeprecationWarning: NotImplemented should not be used in a boolean context
2024-11-14 11:46:42 - WARNING - DeprecationWarning: NotImplemented should not be used in a boolean context
2024-11-14 11:46:42 - WARNING - DeprecationWarning: NotImplemented should not be used in a boolean context
2024-11-14 11:46:42 - WARNING - DeprecationWarning: NotImplemented should not be used in a boolean context
2024-11-14 11:46:42 - WARNING - DeprecationWarning: NotImplemented should not be used in a boolean context
2024-11-14 11:46:42 - WARNI

Traceback (most recent call last):
  File "/home/thompsong/miniconda3/envs/passoft3/bin/rt2ms", line 8, in <module>
    sys.exit(main())
  File "/home/thompsong/miniconda3/envs/passoft3/lib/python3.10/site-packages/rt2ms_py3/rt2ms_py3.py", line 191, in main
    parameters.explore(cf_dirs)
  File "/home/thompsong/miniconda3/envs/passoft3/lib/python3.10/site-packages/rt2ms_py3/parameter_file.py", line 125, in explore
    rt130_soh = rt130_file.get_reftek130_from_file_soh(rt130_file.path)
  File "/home/thompsong/miniconda3/envs/passoft3/lib/python3.10/site-packages/rt2ms_py3/rt130_files.py", line 79, in get_reftek130_from_file_soh
    rt._data = _initial_unpack_packets_soh(string)
  File "/home/thompsong/miniconda3/envs/passoft3/lib/python3.10/site-packages/obspy_replace/soh_packet.py", line 935, in _initial_unpack_packets_soh
    result['time'][:] += [_get_nanoseconds_for_start_of_year(y)
  File "/home/thompsong/miniconda3/envs/passoft3/lib/python3.10/site-packages/obspy_replace/soh_pack

## 2.2 Parse 1/* and 9/* files to create a dataframe of datalogger/yyyyjjj + station/lat/lon history

In [ ]:
#ServiceRunDirs = sorted(glob.glob(os.path.join(REFTEKDIR, 'CONVERT', 'SVC??')))
ServiceRunDirs = [NEWSVCDIR]
for SVCDIR in ServiceRunDirs:

    # Create summary files from 1/* and 9/* files
    summary1file, positionDF = libGTpasscal.create_summary1file(SVCDIR, newpaths['RAWDIR'])
    display(positionDF)
    """
    if len(positionDF)>0:
        summary9file, combinedDF = create_summary9file(SVCDIR, newpaths['RAWDIR'], positionDF)
        if not combinedDF.empty:
            display(combinedDF)
            # march through the dataframe chronologically, finding any statistically significant changes in GPS coordinates from one day to another for same digitizer-station combo
        else:
            display(positionDF)
            # march through the dataframe chronologically, finding any changes from one day to another in digitizer-station combo
    """

In [16]:
digitizers = positionDF['Digitizer'].unique()
print(digitizers)
STATIONS = ['BHP1', 'BHP2', 'BHP3', 'BHP4', 'BHP5', 'BHP6', 'BHP7', 'BHP8', 'TANK', 'FIRE', 'BCHH', 'DVEL', 'RBLAB']

['91F8' '92B7' '9338' '9406' '98E6' '9BC5' '9D7C' 'AB13']


In [ ]:
for digitizer in digitizers:
    digitizerDF = positionDF[positionDF['Digitizer']==digitizer]
    stationlists = digitizerDF['Station'].unique()
    for stationliststr in stationlists:
        stations = []
        print(stationliststr)
        #print(type(stationlist))
        for sta in STATIONS:
            if stationliststr.find(sta)>-1:
                stations.append(sta)
        #stations = stationlist.replace("[","").replace("]","").replace("\\","") 
        #split("\"") #.split(",").split("[").split("]")
        print(digitizer, len(stations), stations)
        

    

In [ ]:
position2DF = positionDF.copy()
for i, row in position2DF.iterrows():
    stations = []
    for sta in STATIONS:
        if row['Station'].find(sta)>-1:
            stations.append(sta)
    position2DF.at[i, 'Station'] = stations
print(position2DF)

In [27]:
position2DF.to_csv(summary1file.replace('.csv', 'new.csv'), index=False)

In [26]:
display(position2DF)

,Digitizer,DateTime,Station,Latitude,Longitude,Latitude2,Latitude2std
0,91F8,2018-10-15 22:03:40,[BHP2],NaN,NaN,28.06038,0.00000
1,91F8,2018-10-15 23:00:00,[BHP2],28.06015,-82.40087,28.06038,0.00000
2,91F8,2018-10-15 23:46:40,[BHP2],28.06015,-82.40087,NaN,NaN
3,91F8,2018-10-16 00:00:00,[BHP2],28.06025,-82.40088,28.06038,0.00000
4,91F8,2018-10-16 01:00:00,[BHP2],28.06055,-82.40100,28.06038,0.00000
...,...,...,...,...,...,...,...
61994,AB13,2021-02-05 08:00:00,[FIRE],28.55003,-80.61845,28.55004,0.00000
61995,AB13,2021-02-05 09:00:00,[FIRE],28.55008,-80.61845,28.55007,0.00000
61996,AB13,2021-02-05 10:00:00,[FIRE],28.55010,-80.61845,28.55004,0.00001
61997,AB13,2021-02-05 11:00:00,[FIRE],28.55003,-80.61845,28.55007,0.00000


In [30]:
import pandas as pd
lastdig = ''
laststa = ''
lastrow = {}
lasti = -1
lod = []
for i, row in position2DF.iterrows():
    if row['Digitizer']!=lastdig:
        if len(lastrow)>0:
            if i > lasti+1:
                #print(lastrow)
                lod.append(lastrow)
        #print(row)
        lod.append(row)
        lasti = i
    elif row['Station']!=laststa:
        #print(row)
        lod.append(row)
        lasti = i
    lastdig = row['Digitizer']
    laststa = row['Station']
changeDF = pd.DataFrame(lod)
display(changeDF)

,Digitizer,DateTime,Station,Latitude,Longitude,Latitude2,Latitude2std
0,91F8,2018-10-15 22:03:40,[BHP2],NaN,NaN,28.06038,0.00000
2453,91F8,2019-01-25 21:00:00,"[BHP2, BCHH]",28.57427,-80.57287,28.57424,0.00001
2454,91F8,2019-01-25 22:00:00,[BHP2],28.57425,-80.57288,28.57424,0.00001
12647,92B7,2018-08-10 21:56:36,[BCHH],28.57347,-80.57233,NaN,NaN
12963,92B7,2018-12-04 23:18:21,"[BHP1, BCHH]",28.57348,-80.57232,28.57347,0.00000
...,...,...,...,...,...,...,...
59671,AB13,2020-10-15 14:00:00,[FIRE],28.55005,-80.61843,28.55004,0.00004
60403,AB13,2020-11-15 01:00:00,"[BHP5, FIRE]",28.55007,-80.61848,28.55006,0.00001
60404,AB13,2020-11-15 02:00:00,[FIRE],28.55005,-80.61847,28.55003,0.00003
60929,AB13,2020-12-23 00:00:00,"[BHP2, FIRE]",28.55005,-80.61847,28.55006,0.00006


In [31]:
changeDF.to_csv(summary1file.replace('summary', 'change'), index=False)